# CoT (Chain of Thought)

In [1]:
from dotenv import load_dotenv
import openai

load_dotenv()
client = openai.OpenAI()

### 일반 프롬프트 vs Zero-shot CoT

In [ ]:
# 일반 프롬프트
prompt = "123 x 31 + 0.123 x 0.0012는 얼마인가요?"

response = client.chat.completions.create(
    model = 'gpt-4', # 최신 모델은 단계적 추론을 알아서 하기 때문에 낮은 모델로 테스트
    messages=[{'role':'user', 'content':prompt}]
)

In [8]:
print(response.choices[0].message.content)

3813.0000001464입니다.


In [7]:
# CoT 프롬프트
prompt = "단계별로 계산하세요: 123 x 31 + 0.123 x 0.0012는 얼마인가요?"

cot_response = client.chat.completions.create(
    model = 'gpt-4', # 최신 모델은 단계적 추론을 알아서 하기 때문에 낮은 모델로 테스트
    messages=[{'role':'user', 'content':prompt}]
)

In [9]:
print(cot_response.choices[0].message.content)

123 x 31 = 3813
0.123 x 0.0012 = 0.0001476

그래서, 3813 + 0.0001476 = 3813.0001476입니다.


### Few-shot CoT

In [10]:
# 일반 프롬프트
prompt = "123 x 31은 얼마인가요?"

normal_response = client.chat.completions.create(
    model = 'gpt-4', # 최신 모델은 단계적 추론을 알아서 하기 때문에 낮은 모델로 테스트
    messages=[{'role':'user', 'content':prompt}]
)

In [11]:
print(normal_response.choices[0].message.content)

123 x 31은 3813입니다.


In [28]:
# Few-shot CoT
few_shot_cot_prompt = """ 
Q: 123 x 31은 얼마인가요?
1. 123을 분해합니다: 123 = 100 + 20 + 3
2. 각 항을 31과 곱합니다:
    - 100 x 31 = 3100
    - 20 x 31 = 620
    - 3 x 31 = 93
3. 이제 이 결과들을 더합니다:
    - 3100 + 620 + 93
4. 계산을 진행하면:
    - 3100 + 620 = 3720
    - 3720 + 93 = 3813

A: 따라서 123 x 31 = 3813 입니다.

Q: 789 x 56은 얼마인가요? 위와 같이 단계적으로 계산하세요
A:
"""

few_shot_cot_response = client.chat.completions.create(
    model = 'gpt-4', # 최신 모델은 단계적 추론을 알아서 하기 때문에 낮은 모델로 테스트
    messages=[{'role':'user', 'content':few_shot_cot_prompt}]
)

In [29]:
print(few_shot_cot_response.choices[0].message.content)

1. 789을 분해합니다: 789 = 700 + 80 + 9
2. 각 항을 56과 곱합니다:
    - 700 x 56 = 39200
    - 80 x 56 = 4480
    - 9 x 56 = 504
3. 이제 이 결과들을 더합니다:
    - 39200 + 4480 + 504
4. 계산을 진행하면:
    - 39200 + 4480 = 43680
    - 43680 + 504 = 44184

A: 따라서 789 x 56 = 44184 입니다.


### 논리 문제 해결

In [30]:
prompt = """ 
앨리스(Alice)는 밥(Bob)보다 나이가 많다.
밥(Bod)은 찰리(Charlie)보다 나이가 많다.
데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily) 보다는 많다.
찰리(Charlie)는 프랭크(Frank)보다 많다.
에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다 어리다.
프랭크(Frank)는 데이비드(David)보다 어리다.
그렇다면 가장 어린 사람은 누구인가요?
"""

In [31]:
response = client.chat.completions.create(
    model='gpt-4',
    messages=[{'role':'user', 'content':prompt}]
)

In [32]:
response.choices[0].message.content

'가장 어린 사람은 프랭크(Frank)입니다.'